In [1]:
import os
import sys
import site
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, PaliGemmaProcessor, AutoModelForPreTraining
from huggingface_hub import login
import torch
from PIL import Image
import requests

In [2]:
os.system(f"{sys.executable} -m pip install huggingface-hub Pillow")

0

In [3]:
python_version = f"{sys.version_info.major}.{sys.version_info.minor}"
site_packages_path = os.path.expanduser(f'~/.local/lib/python{python_version}/site-packages')
site.addsitedir(site_packages_path)

In [4]:
from huggingface_hub import notebook_login

notebook_login()

In [5]:
# Load PeftConfig and base model
config = PeftConfig.from_pretrained("vannynakamura/paligemma_vqav2")
base_model = AutoModelForPreTraining.from_pretrained("google/paligemma-3b-pt-224")
model = PeftModel.from_pretrained(base_model, "vannynakamura/paligemma_vqav2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/62.6k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/1.74G [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/45.3M [00:00<?, ?B/s]

In [10]:
# Loading and Processing the Image
input_text = "<image> <bos> What is in this image?"
img_url = "https://datasets-server.huggingface.co/assets/merve/vqav2-small/--/4b070c6254225a7355070c94e14c3275606f521d/--/default/validation/25/image/image.jpg?Expires=1732217432&Signature=lJBH-mCQ8GzRnqM75rup4pR-hfeWuhhcq-0MkZCClaHqv8bhoN~h4iiTN11Xf5mYvZ5osshYrsvAOdsjhk0c55-Vfe-p-emCO8by0d3TX8~8COAyXJ6RhB0lMPt4uee8tvvXpwTxtJ~YQXuYwFY9ZpcgVqsb2HSuVIcB5YPRtZg4tJymg1O9zjTBDFRrV60pbny7eJhdJ4ZPBP5baEJhYP1Q322XTVhxqbs9zzWEbvO7ykiRfMIURRQzM1-hweX94M-7bnLTf1gpGM-lwp554PeI57gVLOX6iLOHeoF1loUttqQmEjk0ZMYKvZssfqGAN17WW8sCl3~LwL7POAiw9Q__&Key-Pair-Id=K3EI6M078Z3AC3"
input_image = Image.open(requests.get(img_url, stream=True).raw)

In [11]:
# Loading PaliGemma Processor
processor = PaliGemmaProcessor.from_pretrained("google/paligemma-3b-pt-224")

In [12]:
# Preprocessing Inputs
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
inputs = processor(text=input_text, images=input_image, padding="longest", do_convert_rgb=True, return_tensors="pt").to(device)
model.to(device)
inputs = inputs.to(dtype=model.dtype)

In [13]:
with torch.no_grad():
    output = model.generate(**inputs, max_length=496)

print(processor.decode(output[0], skip_special_tokens=True))

  What is in this image?
dog, ocean, sand, water.
